In [ ]:
import os
import glob
import shutil
import hashlib
from collections import OrderedDict
import pandas as pd

HEADER_ROW=['bag', 'time_sec', 'time_nsec', 'GPS', 'v_ego', 'key_event', 'frame_name']
frames_dir = '/media/dean/deans_data/construction_frames/'
bag_dirs = os.listdir(frames_dir)
save_dir = os.path.join(frames_dir,'construction_set/')
os.makedirs(save_dir, 0o755 , exist_ok = True )

img_data = []
lookup_table = {}

for bag in bag_dirs:
    img_dict = {}
    bag_path = os.path.join(frames_dir, bag)
    img_dict['bag_path'] = bag_path
    img_dict['frames'] = []
    img_dict['csv_path'] = os.path.join(bag_path, 'frames.csv')
    csv_logger = pd.read_csv(img_dict['csv_path'], names=HEADER_ROW, skiprows=1)
    
    # Get all images
    imgs = glob.glob(os.path.join(bag_path,'**/', '*.jpg'))
    
    for img in imgs:
        d = {}
        d['frame_path'] = img
        new_path = os.path.split(os.path.split(os.path.split(img)[0])[0])[1].rstrip('/')+'-'+os.path.split(img)[1]
        hash_object = hashlib.sha1(str.encode(new_path))
        hex_dig = hash_object.hexdigest()
        shutil.copyfile(img, os.path.join(save_dir, str(hex_dig)+'.jpg'))
        d['dataset_path'] = os.path.join(save_dir, str(hex_dig)+'.jpg')
        d['hash_key'] = hex_dig
        d['hash_val'] = new_path
        lookup_table[d['hash_key']] = d['hash_val']    
        
        for log in [x for x in csv_logger.as_matrix() if os.path.join(bag_path, x[6]) == d['frame_path']]:
            d['bag_name'] = log[0]
            d['time_sec'] = log[1]
            d['time_nsec'] = log[2]
            d['GPS'] = log[3]
            d['v-ego'] = log[4]
            d['key_event'] = log[5]
            d['frame'] = log[6]
        
        img_dict['frames'].append(d)
    img_data.append((bag_path,img_dict))
    
print(img_data)
pickle_file = os.path.join(save_dir, 'kacheai_logs'+'.pickle')
pickle_dict = {'img_data':img_data,'lookup_table':lookup_table}
print('Saving to Pickle File:', pickle_file)
with open(pickle_file,"wb") as pickle_out:
    pickle.dump(pickle_dict, pickle_out)



/home/dean/miniconda3/envs/ros-kache/lib/python3.6/site-packages/ipykernel/__main__.py:40: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [16]:
for lst in remove_list:
    try:
        os.remove(os.path.join(save_dir,lst))
    except OSError:
        pass